# Part 3: Analyzing audio signals

(Draft) 

In this part, we consider the inverse problem of what we've been dealing with so far -- given a digitized audio signal that is known to be a sine wave at some particular constant frequency, our task is to find out what that frequency is. The approach we'll take here is not one of mathematical rigour, but by working our way up gaining an experimental understanding of the issues.

## Preliminaries

What should we consider to be the "average value" of a sine wave in the interval $[0,T]$? It can be expressed as an integral 

$$\text{average}\{\sin(2\pi f t + \phi)\} = (1/T)\int_0^T{\sin(2\pi f t + \phi)dt}$$ 

We can calculate this integral exactly as $$\frac{(\cos(\phi)-\cos(2\pi f T + \phi))}{2\pi fT}$$. We left a phase value $\phi$ in there to capture the fact that we won't know the phase of the given sine wave and will have to account for that. Note that if we average over an exact number of periods, we get 0 obviously.

As we increase T, we can see that the numerator stays within the range $[0,2]$ whereas the denominator increases monotonically. Therefore as T increases in such a manner that $2\pi f T \gg 1$, the above expression for the average drops down to $0$. That inequality can be seen to mean "we should average over a sufficient number of periods of the sine wave". Note that if $T$ where such that $fT$ is an integer, then the integral would be exactly $0$ irrespective of $\phi$.

Our conclusion from that assessment is that when averaged over a sufficient number of periods, a sine wave averages to $0$.

Now let's consider the "amplitude modulated sine wave" -- $\text{sin}(2\pi ft + \phi)\text{sin}(2\pi f't)$. We use $\phi$ to capture the fact that we may have an unknown phase difference between the two sines. We'll ignore the amplitude for now in all of this since it is only a constant of proportionality.

From basic trigonometry, we know that 

$$\text{cos}(A-B) - \text{cos}(A+B) = 2\text{sin}(A)\text{sin}(B)$$

.. which gives us --

$$\text{sin}(2\pi ft + \phi)\text{sin}(2\pi f't) = (1/2)(\text{cos}(2\pi(f'-f)t - \phi) - \text{cos}(2\pi(f'+f)t + \phi))$$

When averaging the above expression, let's use $\delta f = f' - f$. So we have the average over a time $T$ as -

$$\frac{1}{2}(\frac{\sin(2\pi \delta f T - \phi) + \sin(\phi)}{2\pi \delta fT} - \frac{\sin(2\pi(2f +\delta f)T + \phi) - \sin(\phi)}{2\pi(2f+\delta f)T})$$

Since we generally expect $\delta f << f$, and $fT >> 1/2$, we can ignore the second term as averaging out to zero. That leaves us with -

$$\text{c}(\delta fT,\phi) = \frac{1}{2}\frac{\sin(2\pi\delta fT)\cos(\phi) - (\cos(2\pi\delta fT)-1)\sin(\phi)}{2\pi\delta fT}$$

If we then consider -

$$\text{c'}(\delta fT) = c(\delta fT,\phi)^2 + c(\delta fT,\phi+\pi/2)^2$$

we can see that $c'$ is independent of $\phi$, which is an unknown phase difference between the given sine wave and the sine wave we correlate it with. This function is pretty flat around $\delta fT \approx 0$, which suggests that it is not useful to pick an arbitrarily small value for $\delta f$ to test since we won't be able to tell much of the difference between $\delta f$ and say $2\delta f$, especially since real world signals won't be pure sine waves. This holds mostly up to about $\delta fT \approx 1/4$. So if we choose a $\delta f$ that's too small, we'll anyway have difficulty telling which of the various $k\delta f$ for integer $k$ we should take as the peak. It doesn't help therefore to pick a $\delta f$ that's less than about $1/{4T}$. This further means that if we want a frequency measurement that's accurate to $\delta f$, then we should average over a sufficiently long period of time determined by $T \approx 1/\delta f$. Note that this means we don't know the time at which the signal takes on the found frequency within an interval of span $T$. So you can only find out one of the two - frequency or time - to arbitrary precision. This is the "frequency - time uncertainty principle" in action.

We therefore now have the beginning of an algorithm to find the frequency of a given sine wave -- multiply it by two sine waves with a phase difference of $\pi/2$, of a chosen test frequency, average the compound signal and see what we get. We try various frequencies until we find where we get the largest value for the average. For simplicity, we can pick a $\delta f$ according to the above logic and test all the frequencies $f' = k\delta f$ for integer $k$.

## The "spectrum"

We've seen how a given simple audio waveform like "sine wave" can be described in two different ways - by giving how it varies in time, as well as what its "frequency" is. If we mix multiple such sine wave (by adding the signals), then we can equivalently describe the mixture by giving the freuqencies we used and the relative strengths of those components.

In the above workout, we calculated two "averages" with sine waves that are apartby $90^\circ$ and combined them using $\sqrt{x^2+y^2}$ to cancel out the unknown phase. We can generalize that using the complex signal $\omega(f,t) = e^{-2\pi i ft} = \cos(2\pi ft) - i\sin(2\pi ft)$. We denote the conjugate as 

$$\omega^\star(f,t) = e^{2\pi ift} = \cos(2\pi f t) + i\sin(2\pi ft) = \omega(-f,t) = \omega(f,-t)$$

If for a given sound signal $s(t)$ we consider the average over all time -

$$S(f) = \int_{-\infty}^{+\infty}\omega(f,t)s(t)dt = \int_{-\infty}^{+\infty}e^{-2\pi ift}s(t)dt$$

the function $S(f)$ can equivalently describe the signal $s(t)$ in terms of the frequencies it is composed of. To recover $s(t)$ given $S(t)$, we can do -

$$s(t) = \int_{-\infty}^{+\infty}\omega^\star(f,t)S(f)df = \int_{-\infty}^{+\infty}e^{2\pi ift}S(f)df$$

The signal expression $s(t)$ is known in literature as a **time domain representation** and its corresponding function $S(f)$ is called its **frequency domain representation** or the **spectrum** or the **Fourier transform** of $s(t)$.

For now, just knowing that we can think in terms of these two representations will suffice and we'll work through some of these aspects as we go along in more detail.


## A couple of important interesting facts about the spectrum

One interesting aspect worth pointing out is that if we time stretch the time domain signal $s(t)$ to be $s(\alpha t)$, then its "spectrum" gets shrunk by $\alpha$ - i.e. the spectrum of $s(\alpha t)$ is $S(f/\alpha)$. Similarly, if we shrink $s(t)$, its spectrum gets stretched by the same factor. You can use this result to see why you cannot measure both the precise frequency of a pure tone signal *and* the precise time at which it occurs (when it is time limited) -- because if you shrink the time extent of the signal, the spectrum gets more spread out and vice versa.

Similarly if we time shift $s(t)$ to $s(t-t_0)$, then the spectrum gets multiplied by a complex phase -- i.e. the spectrum of $s(t-t_0)$ is $e^{-2\pi ift_0}S(f)$. This also holds in reverse - i.e. if you "frequency shift" $S(f)$ to $S(f-f_0)$, the corresponding time domain signal becomes modulated by a cos-sin wave $e^{2\pi if_0t}s(t)$.

If you consider $f(t) = \int_{-\infty}^{+\infty}{e^{2\pi ift}F(f)df}$, it is easy to see how $$f'(t) = \int_{-\infty}^{+\infty}{(2\pi if)e^{2\pi ift}F(f)df}$$ -- i.e. the Fourier transform of $f'(t)$ is simply $F(f)$ multiplied by $2\pi if$. This property makes the Fourier transform (and the related "Laplace Transform") very useful in solving linear differential equations.

## Task

Given a sine wave signal in the form we've been using so far, can you code an algorithm that will find the frequency of the sine using the above approach?